# 04 Full PM Simulation (Package-backed)

This notebook runs the modular simulation pipeline end-to-end, then validates and visualizes the outputs. It replaces the monolithic notebook implementation with calls to the package API.

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / "src").exists() and (ROOT.parent / "src").exists():
    ROOT = ROOT.parent
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
print("Using src path:", SRC)

In [ ]:
from pathlib import Path
from lcdm_sim.config import simulation_config_from_dict
from lcdm_sim.simulation import run_simulation
from lcdm_sim.validation import run_validation_suite
from lcdm_sim.plotting_static import plot_grf_slice, plot_density_projection
from lcdm_sim.plotting_interactive import plot_evolution_summary_interactive

cfg = simulation_config_from_dict({
    "grid": {"n_particles_1d": 16, "n_grid_1d": 16, "box_size_mpc_h": 50.0},
    "cosmology": {"h0": 67.66, "omega_m": 0.3097, "omega_lambda": 0.6903, "sigma8": 0.3, "n_s": 0.96, "a_initial": 0.1, "a_final": 0.2},
    "integrator": {"num_steps": 8, "method": "kdk_a"},
    "output": {"output_root": "outputs", "save_density": True, "save_plots": False},
    "performance": {"fft_backend": "scipy", "use_numba": False, "fft_workers": 1},
    "validation": {"enable_invariants": True, "enable_reference_compare": False},
    "random_seed": 7
})
outdir = ROOT / "outputs" / "notebook_demo_run"

In [ ]:
result = run_simulation(cfg, output_dir=outdir, num_snapshots=4, history_stride=2, save_snapshots=True)
report = run_validation_suite(result, cfg)
print("Run ID:", result.run_id)
print("Snapshots:", [s.step for s in result.snapshots])
print("Validation ok:", report.ok, report.summary)

In [ ]:
plot_dir = outdir / "plots"
plot_dir.mkdir(parents=True, exist_ok=True)
_ = plot_grf_slice(result.initial_density, plot_dir / "grf_slice.png")
if result.snapshots and result.snapshots[-1].density_field is not None:
    _ = plot_density_projection(result.snapshots[-1].density_field, plot_dir / "final_projection.png")
print("Static plots written to", plot_dir)

In [ ]:
history_html = plot_evolution_summary_interactive(result.history, outdir / "plots" / "evolution_summary.html")
history_html

## Notes

- The simulation core is now modular and testable.
- Validation is run programmatically via `lcdm_sim.validation`.
- Plotting helpers are reusable from scripts, tests, and notebooks.